## Basic atomese examples

In [ ]:
import common
from opencog.utilities import initialize_opencog
from opencog.atomspace import AtomSpace, types
from opencog.type_constructors import *
from opencog.bindlink import bindlink
from opencog.bindlink import execute_atom

Make a new atomspace  
Atomspace is a hypergraph database  
which can store nodes, links and associated values

In [ ]:
atomspace = AtomSpace()

add concept node  
Nodes destingueshed by type and name and are immutable

In [ ]:
atomspace.add_node(types.ConceptNode, "small")

set default atomspace to add atoms to

In [ ]:
initialize_opencog(atomspace)

adding node one more time doesn't modify anything

In [ ]:
ConceptNode("small")

every Atom(nodes or links) has incoming and outgoing sets

In [ ]:
print('ConceptNode("size").incoming:')
print(ConceptNode("size").incoming)

every Atom has default tv:

In [ ]:
link = InheritanceLink(ConceptNode("small"), ConceptNode("size"))
print("create link {0}\n".format(link))

print(link.tv)

setting new TruthValue

In [ ]:
link.tv = TruthValue(0.9, 0.9)

Outgoing set of a link is set of Atoms it connects

In [ ]:
print(link)
print('outgoing set of the link:')
print(link.out)

Let's put the InheritanceLink into a ListLink  
ListLink is just ordered List of atoms

In [ ]:
ListLink(link)

In [ ]:
print('incoming set of the link:')
print(link.incoming)

While incoming set of Atom is set of links pointing to it

In [ ]:
print('\n incoming set of ConceptNode("size")')
print(ConceptNode("size").incoming)

Thus outgoing set of Nodes are empty

In [ ]:
ConceptNode("size").out

Let's take a look at BindLink  
Semantics of BindLink is  
"do something if pattern matches and return the result"

**Bindlink structure**
1. VariableList - optional  
2. pattern to match - small atomspace subgraph possibly     
   containing variables 
3. Rewrite term 

If pattern doesn't contain variable you already know the result

Our condition will be InheritanceLink between  two variables  
So pattern matcher will try to replace VariableNodes with other Atoms

In [ ]:
pattern = InheritanceLink(VariableNode("X"), VariableNode("Y"))

Let's return the same link which is matched in pattern

In [ ]:
bindlink1 = BindLink(pattern, pattern)
print(str(bindlink1))

BindLink constructor doesn't execute anything  
There is interface function for pattern matcher  
"bindlink" which accepts atomspace to apply bindlink to  
and bindlink itself

In [ ]:
print("running bindlink")
print("result: \n{0}".format(bindlink(atomspace, bindlink1)))

______________________________________________________
Another usefull type of link is ExecutionOutputLink  
which used to run callback function on matched pattern  
ExecutionOutputLink should return Atom


**ExecutionOutputLink structure**
1. GroundedSchemaNode
2. ListLink of arguments

let's define a callback function which returns a ConceptNode

In [ ]:
def add_node():
    return ConceptNode("Test")

Now GroundedSchemaNode which accepts string of following format  
"&lt;language&gt;:&lt;function name&gt;"


where language can be any of "lib", "py", "scm"

In case of lib atomspace will try to open library with dlopen  
so any shared library may be used

In [ ]:
add_node_schema = GroundedSchemaNode("py: add_node")

Now make Execution output link

In [ ]:
exec1 = ExecutionOutputLink(
         add_node_schema,
         ListLink())


ExecutionOutputLink may be used inside of bindlink or  
it may be executed by execute_atom function

In [ ]:
print("running ExecutionOutputLink")
print(execute_atom(atomspace, exec1))

ExecutionOutputLinks may be nested(actually any links may)

In [ ]:
print("nested ExectutionOutputLinks")

def add_link(node):
    ev = EvaluationLink(
           PredicateNode("ok"),
           node)
    ev.tv = TruthValue(0.1, 0.8)
    return ev

exec2 = ExecutionOutputLink(
           GroundedSchemaNode("py: add_link"),
           ListLink(
             ExecutionOutputLink(
                 GroundedSchemaNode("py: add_node"),
                 ListLink()))
        )

print(execute_atom(atomspace, exec2))



Exercise: print incoming set of (InheritanceLink (ConceptNode "small") (ConceptNode "size") )  
once more time